In [1]:
import os 
import glob
from dotenv import load_dotenv
from openai import OpenAI
from langchain_chroma import Chroma
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.callbacks import StdOutCallbackHandler
import gradio as gr

/Users/saud/Desktop/LLM_Projects/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [3]:
MODEL = 'gpt-4o-mini'
vector_db = 'vector_db'

In [4]:
documents = []

folders = glob.glob('knowledge-base/*')
for folder in folders:
    doc_type = folder.split(os.sep)[-1]
    loader = DirectoryLoader(folder, glob= "**/*.md", loader_cls= TextLoader)
    folder_docs = loader.load()
  
    for doc in folder_docs:
        doc.metadata['doc_type'] = doc_type
        documents.append(doc)

In [5]:
text_splitter = CharacterTextSplitter(chunk_size = 1088, chunk_overlap = 200)
spllited_text = text_splitter.split_documents(documents)

In [6]:
doc_types = set(chunk.metadata['doc_type'] for chunk in spllited_text)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: company, employees, products, contracts


In [7]:
embeddings = OpenAIEmbeddings()

In [ ]:
# if os.path.exists(vector_db):
#     Chroma(persist_directory= vector_db, embedding_function= embeddings).delete_collection()

In [9]:
# vectorStore = Chroma.from_documents(embedding= embeddings, persist_directory= vector_db, documents= spllited_text)
# print(f"Vector Store created with {vectorStore._collection.count()} documents.")

In [10]:
# collection = vectorStore._collection
# result = collection.get(limit=1, include=['embeddings'])['embeddings'][0]
# dimensions =  len(result)
# print(f"The vector has {dimensions} dimensions.")


In [11]:
vectorStore = FAISS.from_documents(embedding= embeddings, documents= spllited_text)
print(f"Vector Store created with {vectorStore.index.d} dimensions and {vectorStore.index.ntotal} documents")

Vector Store created with 1536 dimensions and 109 documents


In [ ]:
def RAG_pipeline(prompt, history):
    llm = ChatOpenAI(temperature= 0.5, model= MODEL)
    retriever = vectorStore.as_retriever(search_kwargs = {"k": 20})
    memory = ConversationBufferMemory(memory_key= 'chat_history', return_messages= True)

    conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

    return conversation_chain.invoke({'question': prompt, "chat_history": history})['answer']

In [13]:
gr.ChatInterface(RAG_pipeline).launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [14]:
def RAG_testing_pipeline(prompt, history):
    llm = ChatOpenAI(temperature= 0.5, model= MODEL)
    retriever = vectorStore.as_retriever()
    memory = ConversationBufferMemory(memory_key= 'chat_history', return_messages= True)

    conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks= [StdOutCallbackHandler()])

    return conversation_chain.invoke({'question': prompt, "chat_history": history})['answer']

In [15]:
RAG_testing_pipeline('Who won the prestigious IIOTY award in 2023?', [])



> Entering new ConversationalRetrievalChain chain...


/var/folders/l7/3c4w5v3147lfbftqk954bsxh0000gn/T/ipykernel_75915/2781222458.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key= 'chat_history', return_messages= True)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
## Annual Performance History
- **2019:** Exceeds Expectations - Continuously delivered high-quality code and participated actively in team meetings.
- **2020:** Meets Expectations - Jordan K. Bishop maintained steady performance but faced challenges due to a higher workload from multiple projects.
- **2021:** Exceeds Expectations - Recognized for leadership during the customer portal project; received the “Innovation Award” for creative problem-solving.
- **2022:** Meets Expectations - While mentoring others, the shift in focus led to fewer contributions to new features, marking a decrease in performance.
- **2023:** Needs Improvement - Transitioning back to development has resulted in difficulties 

"I don't know."